In [1]:
import folium
import pandas as pd
import gpxpy
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import numpy as np
import collections
%matplotlib notebook


In [2]:
gpx = gpxpy.parse(open('activity_4257789098.gpx')) # a gpx file from my garmin watch for a run I did
track = gpx.tracks[0]
segment = track.segments[0]

In [5]:
data = []
segment_length = segment.length_3d()

extension_data = collections.defaultdict(list)
for point_idx, point in enumerate(segment.points):
    extensions = point.extensions

    for child in extensions[0].getchildren(): # this gets the extension data (hr and cadence) from the gpx file
        tag = child.tag.rsplit("}", 1)[-1]
        value = child.text
        extension_data[tag].append(float(value))
            

    data.append([point.longitude, point.latitude,
                 point.elevation, point.time, segment.get_speed(point_idx)])
    

columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = pd.DataFrame(data, columns=columns)
df['points'] = list(zip(df.Latitude, df.Longitude))

df.Time = pd.to_datetime(df.Time, format = "%Y-%m-%d %H:%M:%S")
df.Time = df.Time.dt.tz_localize(None)
df.Time = (df.Time- df.Time.values[0]).dt.total_seconds()

df['hr'] = extension_data['hr']
df['cad'] = extension_data['cad']
df['hr'] = df.hr.rolling(25, center = True).mean() # smooth the heart rate
df = df.dropna()
df.head()

,Longitude,Latitude,Altitude,Time,Speed,points,hr,cad
12,-111.009704,45.672581,1487.199951,18.0,3.289993,"(45.672580590471625, -111.00970432162285)",110.64,82.0
13,-111.009804,45.672635,1488.199951,21.0,3.357946,"(45.67263473756611, -111.0098035633564)",112.36,82.0
14,-111.009874,45.672672,1488.000000,23.0,3.400460,"(45.67267161794007, -111.00987430661917)",113.92,82.0
15,-111.009977,45.672727,1487.599976,26.0,3.313763,"(45.672727189958096, -111.00997681729496)",115.56,83.0
16,-111.010047,45.672760,1487.599976,28.0,3.347369,"(45.6727597117424, -111.01004655472934)",117.12,83.0


In [13]:
bzn = [45.6770, -111.0429]

loc = [df.Latitude.mean(), df.Longitude.mean()]



m = folium.Map(location = loc,zoom_start=14)
def rgb2hex(rgb):
    rgb = [hex(int(256*x)) for x in rgb]
    r, g, b = [str(x)[2:] for x in rgb]
    return "#{}{}{}ff".format(r, g, b)

norm = mpl.colors.Normalize(vmin=60, vmax=max(df.hr.values))
color_mapper = cm.ScalarMappable(norm = norm,cmap=cm.jet)
rgb_vals = []
for v in df.hr.values:
    rgb_vals.append(list(color_mapper.to_rgba(v)[0:3]))

colors = [rgb2hex(rgb) for rgb in rgb_vals]

df['Colors'] = colors

for i, row in df.iterrows():
    folium.CircleMarker(location=row.points,
                        radius=5,
                        popup=str(row.hr),
                        color=row.Colors, fill = True).add_to(m)
    

m    
